In [2]:
import pandas as pd
from pprint import pprint

elec_2022 = pd.read_csv("../dataset/Elections Dataset/pres_precinct_data_2022.csv")
elec_2019 = pd.read_json("../dataset/Elections Dataset/2019/philippine_provinces_cities_municipalities_and_barangays_2019v2.json")
elec_2016 = pd.read_csv("../dataset/Elections Dataset/vp_precinct_data_2016.csv")

In [39]:
column_name_change = {'barangay': 'Barangay', 'province': 'Province', 'region': 'Region', 'municipality': 'City'}
elec_2016.rename(columns=column_name_change, inplace=True)

In [68]:
loc_columns = ['Barangay', 'Province', 'Region', 'City']
loc_2022 = pd.DataFrame(elec_2022[loc_columns].sort_index(axis=1).reset_index(drop=True))
loc_2016 = pd.DataFrame(elec_2016[loc_columns].sort_index(axis=1).reset_index(drop=True))

In [81]:
loc_2016 = loc_2016.reindex_like(loc_2022)

In [83]:
loc_2016.compare(loc_2022) # TODO match the rows, see https://stackoverflow.com/questions/29464234/compare-python-pandas-dataframes-for-matching-rows

Barangay                              City             Province  \
           self                   other      self    other        self   
0       BUENLAG                BALINTAD  CALASIAO  BAUNGON  PANGASINAN   
1       BUENLAG              BUENAVISTA  CALASIAO  BAUNGON  PANGASINAN   
2       BUENLAG                 DANATAG  CALASIAO  BAUNGON  PANGASINAN   
3       BUENLAG                 DANATAG  CALASIAO  BAUNGON  PANGASINAN   
4       BUENLAG                 DANATAG  CALASIAO  BAUNGON  PANGASINAN   
...         ...                     ...       ...      ...         ...   
105661      NaN                    GUGO       NaN    SAMAL         NaN   
105662      NaN  WEST CALAGUIMAN (POB.)       NaN    SAMAL         NaN   
105663      NaN  WEST CALAGUIMAN (POB.)       NaN    SAMAL         NaN   
105664      NaN  WEST DAANG BAGO (POB.)       NaN    SAMAL         NaN   
105665      NaN  WEST DAANG BAGO (POB.)       NaN    SAMAL         NaN   

                    Region              
           other      self       other  
0       BUKIDNON  REGION I    REGION X  
1       BUKIDNON  REGION I    REGION X  
2       BUKIDNON  REGION I    REGION X  
3       BUKIDNON  REGION I    REGION X  
4       BUKIDNON  REGION I    REGION X  
...          ...       ...         ...  
105661    BATAAN       NaN  REGION III  
105662    BATAAN       NaN  REGION III  
105663    BATAAN       NaN  REGION III  
105664    BATAAN       NaN  REGION III  
105665    BATAAN       NaN  REGION III  

[105666 rows x 8 columns]